In [3]:
from __future__ import division
import os
import sys
import cv2
import time
import json
import pydot
import random
import pickle
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from PIL import Image
import tensorflow as tf
from pprint import pprint
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Progbar
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

from utils import vgg as nn
from utils import roi_helpers
from utils import losses as losses_fn
from utils.simple_parser import get_data
from utils import config, data_generators

# 이미지 바운딩 박스 그리기

In [22]:
train_data_path = os.path.join(os.getcwd(), 'save_dota_aug_train')

with open('train_images.txt', 'r') as f:
    train_images = f.readline().split(' ')

In [27]:
for idx, img_name in enumerate(train_images):
    cnt = 0
    img = tf.io.read_file(os.path.join(train_data_path, f"{img_name}_augment.png"))
    img = tf.image.decode_image(img).numpy()
    
    with open(os.path.join(train_data_path, f"{img_name}_augment.txt"), 'r') as f:
        for line in f.readlines():
            cnt += 1
            cls, x1, x2, y1, y2 = line.split(' ')
            x1, x2, y1, y2 = int(float(x1)), int(float(x2)), int(float(y1)), int(float(y2))
            img = cv2.rectangle(img, (x1,y1), (x2,y2), (0,0,255), 2)
        
        
    # plt로 이미지 보기
#     plt.title(f"{img_name}-{cnt:02}")
#     plt.imshow(img)
#     plt.show()



    # 이미지 저장
    cv2.imwrite(os.path.join("visual-dota-aug", f"{img_name}-{cnt}.png"), img)
    if int(img_name[1:]) > 99:
        break
    
    # cv2로 이미지 보기
#     cv2.imshow('img', img)
#     if cv2.waitKey(2000) > 0: # 2초마다 이미지 변경
#         # 아무키나 누르면 종료
#         cv2.destroyAllWindows()
#         break

---

In [7]:
all_img_data = train_images
class_count = train_classes_count
C = cfg
img_length_calc_function = nn.get_img_output_length
mode='train'

## 바운딩 박스 보기

In [8]:
for idx, img_data in enumerate(all_img_data):
    img = tf.io.read_file(img_data['filepath'])
    img = tf.image.decode_image(img).numpy()

    for dt in img_data['bboxes']:
        cls, x1, x2, y1, y2 = list(dt.values())
        img = cv2.rectangle(img, (x1,y1), (x2,y2), (0,0,255), 2)

    print(f"index:{idx}, name:{img_data['filepath'].split('/')[-1][:-4]}, box count:{len(img_data['bboxes'])}")
    cv2.imshow('img', img)

    # 이미지 저장
#     cv2.imwrite(os.path.join("visual-dotat-aug", 
#                              f"{img_data['filepath'].split('/')[-1][:-4]}_boxCount_{len(img_data['bboxes'])}.png"), img)
    
    if cv2.waitKey(2000) > 0: # 2초마다 이미지 변경
        # 아무키나 누르면 종료
        cv2.destroyAllWindows()
        break

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc1 in position 85: invalid start byte

---

In [33]:
for idx, img_data in enumerate(all_img_data):
    print(f'{idx:3}', img_data['filepath'][22:-4], img_data['width'], img_data['height'])

  0 P0000_augment 3875 5502
  1 P0001_augment 1252 5774
  2 P0002_augment 2557 2086
  3 P0005_augment 884 977
  4 P0008_augment 1143 1235
  5 P0010_augment 1846 1865
  6 P0011_augment 1367 1526
  7 P0012_augment 2896 1990
  8 P0013_augment 1338 1776
  9 P0018_augment 1824 1689
 10 P0020_augment 5058 4439
 11 P0021_augment 2870 3234
 12 P0022_augment 3946 6533
 13 P0023_augment 2413 5118
 14 P0025_augment 4567 4101
 15 P0029_augment 2759 2407
 16 P0030_augment 2759 2406
 17 P0032_augment 2425 1689
 18 P0036_augment 3193 5540
 19 P0038_augment 2555 4312
 20 P0039_augment 3599 3656
 21 P0041_augment 3191 2322
 22 P0042_augment 1900 2225
 23 P0044_augment 1648 1732
 24 P0049_augment 1628 1988
 25 P0050_augment 2559 3124
 26 P0052_augment 1861 2273
 27 P0054_augment 2192 2109
 28 P0058_augment 1216 1200
 29 P0061_augment 1954 1818
 30 P0063_augment 2911 2983
 31 P0064_augment 2974 2199
 32 P0065_augment 2380 3406
 33 P0066_augment 3152 3281
 34 P0067_augment 1156 1483
 35 P0068_augment 693 

389 P0759_augment 1139 1085
390 P0760_augment 1439 823
391 P0762_augment 1739 533
392 P0766_augment 939 906
393 P0768_augment 2218 810
394 P0769_augment 1072 1591
395 P0775_augment 1791 1886
396 P0777_augment 1609 1358
397 P0778_augment 1609 1358
398 P0779_augment 659 803
399 P0780_augment 1069 1591
400 P0781_augment 878 1071
401 P0782_augment 878 1071
402 P0783_augment 2388 1364
403 P0785_augment 2111 3433
404 P0786_augment 3344 1911
405 P0790_augment 2344 1637
406 P0792_augment 1388 1088
407 P0794_augment 1647 1405
408 P0800_augment 1576 1912
409 P0803_augment 1450 852
410 P0804_augment 1447 849
411 P0805_augment 1408 1119
412 P0806_augment 1407 1117
413 P0809_augment 1404 1115
414 P0810_augment 963 565
415 P0817_augment 1139 1130
416 P0818_augment 959 583
417 P0819_augment 885 1108
418 P0821_augment 960 584
419 P0823_augment 1745 2473
420 P0825_augment 916 854
421 P0826_augment 636 1082
422 P0827_augment 1054 1887
423 P0828_augment 1516 2193
424 P0829_augment 1561 1685
425 P0831_aug

735 P1470_augment 5503 5660
736 P1475_augment 992 831
737 P1479_augment 2404 2066
738 P1480_augment 2199 2696
739 P1481_augment 2934 3510
740 P1482_augment 2463 2458
741 P1484_augment 1962 2171
742 P1486_augment 1496 1112
743 P1487_augment 1741 1385
744 P1489_augment 4932 4658
745 P1493_augment 4401 5107
746 P1495_augment 4886 5416
747 P1497_augment 3237 4215
748 P1498_augment 2307 3083
749 P1499_augment 2579 2862
750 P1501_augment 1570 3981
751 P1502_augment 4253 4334
752 P1505_augment 2627 2325
753 P1507_augment 1143 1657
754 P1509_augment 4767 3076
755 P1514_augment 4000 4000
756 P1515_augment 4000 4000
757 P1517_augment 4294 4000
758 P1519_augment 4000 4000
759 P1521_augment 4000 4000
760 P1522_augment 4000 2929
761 P1524_augment 4000 4000
762 P1526_augment 4000 4000
763 P1533_augment 4000 4000
764 P1534_augment 4293 4000
765 P1535_augment 4000 4962
766 P1537_augment 4000 4962
767 P1539_augment 4000 4000
768 P1540_augment 4000 4000
769 P1545_augment 4000 4000
770 P1550_augment 4000

1116 P2234_augment 1485 1636
1117 P2235_augment 735 1577
1118 P2237_augment 990 1091
1119 P2240_augment 980 1597
1120 P2243_augment 1997 1669
1121 P2244_augment 1265 1073
1122 P2245_augment 1269 1080
1123 P2247_augment 2798 1167
1124 P2248_augment 759 802
1125 P2249_augment 2306 6222
1126 P2250_augment 3065 1657
1127 P2251_augment 1271 1080
1128 P2253_augment 3314 1219
1129 P2257_augment 732 1081
1130 P2258_augment 2550 911
1131 P2259_augment 1280 512
1132 P2260_augment 1527 597
1133 P2262_augment 2291 893
1134 P2264_augment 2301 626
1135 P2266_augment 730 1082
1136 P2269_augment 1229 1381
1137 P2270_augment 486 805
1138 P2275_augment 3077 2327
1139 P2278_augment 1539 1035
1140 P2279_augment 3337 2585
1141 P2280_augment 4103 5147
1142 P2281_augment 3590 3615
1143 P2282_augment 3076 3093
1144 P2283_augment 2307 2064
1145 P2287_augment 2307 2064
1146 P2288_augment 2824 4883
1147 P2289_augment 993 572
1148 P2301_augment 714 821
1149 P2302_augment 733 563
1150 P2303_augment 476 547
1151 P2

1481 P4330_augment 1024 1024
1482 P4337_augment 1024 1024
1483 P4338_augment 1024 1024
1484 P4350_augment 1024 1024
1485 P4355_augment 1024 1024
1486 P4383_augment 1024 1024
1487 P4400_augment 4096 4096
1488 P4412_augment 1024 1024
1489 P4451_augment 1024 1024
1490 P4464_augment 7360 4912
1491 P4470_augment 1024 1024
1492 P4493_augment 1024 1024
1493 P4513_augment 1024 1024
1494 P4528_augment 1024 1024
1495 P4571_augment 1024 1024
1496 P4577_augment 1024 1024
1497 P4612_augment 1024 1024
1498 P4613_augment 1024 1024
1499 P4625_augment 1024 1024
1500 P4634_augment 1024 1024
1501 P4720_augment 1024 1024
1502 P4724_augment 7360 4912
1503 P4743_augment 1024 1024
1504 P4744_augment 1024 1024
1505 P4746_augment 7360 4912
1506 P4759_augment 1024 1024
1507 P4770_augment 1024 1024
1508 P4803_augment 1024 1024
1509 P4840_augment 1024 1024
1510 P4851_augment 4096 4096
1511 P4914_augment 1024 1024
1512 P4937_augment 1024 1024
1513 P4946_augment 4097 4096
1514 P4987_augment 1024 1024
1515 P4992_aug

In [ ]:
P0906_augment

In [34]:
from utils import data_augment
from utils.data_generators import calc_rpn

img_data = all_img_data[49]

img_data_aug, x_img = data_augment.augment(img_data, C, augment=False)

(width, height) = (img_data_aug['width'], img_data_aug['height'])
(rows, cols, _) = x_img.shape
assert(cols == width)
assert(rows == height)

if width * height > 4000 * 4000:
    resized_width, resized_height = width // 2, height // 2
    x_img = tf.image.resize(x_img, (resized_height, resized_width), 'bicubic')
else:
    resized_width, resized_height = width, height

y_rpn_cls, y_rpn_regr, cache = calc_rpn(C, img_data_aug, width, height, resized_width, resized_height, img_length_calc_function)

x_img /= C.img_scaling_factor
x_img = tf.expand_dims(x_img, axis=0)

y_rpn_regr[:, y_rpn_regr.shape[1]//2:, :, :] *= C.std_scaling

y_rpn_cls = np.transpose(y_rpn_cls, (0, 2, 3, 1))
y_rpn_regr = np.transpose(y_rpn_regr, (0, 2, 3, 1))

# X, Y, img_data, cache = next(data_gen_train)


 ./save_dota_aug_train/P0096_augment.png (1200, 928, 3)
177 128.0 128


In [37]:
y_rpn_regr.shape, 1200 // 16, 928 // 16

((1, 75, 58, 72), 75, 58)

In [38]:
print(y_rpn_regr.shape)
y = y_rpn_regr[0].reshape(y_rpn_regr.shape[1], y_rpn_regr.shape[2], 18, 4)

(1, 75, 58, 72)


In [39]:
y_overlap = y[:, :, :9, :]
y_regr = y[:, :, 9:, :]

print(y_overlap.shape, y_regr.shape)

(75, 58, 9, 4) (75, 58, 9, 4)


In [40]:
H, W, C, _ = np.where(y_overlap == 1)

In [41]:
img = x_img[0].numpy().astype(np.uint8)
print(img.shape)

(1200, 928, 3)


In [49]:
for h, w, c in zip(H, W, C):
    c1,c2,c3,c4 = y_regr[h,w,c]
    print(c1,c2,c3,c4)
    x,y,tw,th = int((w+c1)*16), int((h+c2)*16), int(c3*16), int(c4*16)
    print(h, w, x,y,tw,th)
    print((x-abs(tw),y-abs(th)), (x+abs(tw),y+abs(th)))
    img = cv2.rectangle(img, (x-abs(tw),y-abs(th)), (x+abs(tw),y+abs(th)), (255,0,0), 3)
    break
    
# cv2.imshow('img', img)
cv2.imwrite('visual.png', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

1.6875 1.85546875 -8.317766189575195 -11.484115600585938
62 32 539 1021 -133 -183
(406, 838) (672, 1204)


In [ ]:
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

---

---

In [ ]:
colors = {
    'large-vehicle': (0,0,255),
    'small-vehicle': (255,0,0)
}

In [ ]:
img_data['filepath'].split('/')[-1][:-4]

In [ ]:
img_data = all_img_data[3]

img = tf.io.read_file(img_data['filepath'])
img = tf.image.decode_image(img).numpy()

In [ ]:
for dt in img_data['bboxes']:
    cls, x1, x2, y1, y2 = list(dt.values())
    
    img = cv2.rectangle(img, (x1,y1), (x2,y2), colors[cls], 1)

In [ ]:
# cv2.imshow('img', img)
cv2.imshow(img_data['filepath'].split('/')[-1][:-4], img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
os.path.join("visual-dotat-aug", f"name:{img_data['filepath'].split('/')[-1][:-4]}-box_count:{len(img_data['bboxes'])}")

'visual-dotat-aug\\name:P0000_augment-box_count:279'

- P20은 바운딩 박스가 안보인다??

In [12]:
help(cv2.imwrite)

Help on built-in function imwrite:

imwrite(...)
    imwrite(filename, img[, params]) -> retval
    .   @brief Saves an image to a specified file.
    .   
    .   The function imwrite saves the image to the specified file. The image format is chosen based on the
    .   filename extension (see cv::imread for the list of extensions). In general, only 8-bit
    .   single-channel or 3-channel (with 'BGR' channel order) images
    .   can be saved using this function, with these exceptions:
    .   
    .   - 16-bit unsigned (CV_16U) images can be saved in the case of PNG, JPEG 2000, and TIFF formats
    .   - 32-bit float (CV_32F) images can be saved in PFM, TIFF, OpenEXR, and Radiance HDR formats;
    .     3-channel (CV_32FC3) TIFF images will be saved using the LogLuv high dynamic range encoding
    .     (4 bytes per pixel)
    .   - PNG images with an alpha channel can be saved using this function. To do this, create
    .   8-bit (or 16-bit) 4-channel image BGRA, where the alpha c

In [ ]:
cls, x1, x2, y1,y2

In [ ]:
classes = list()
for dt in img_data['bboxes']:
    cls, x1, x2, y1, y2 = list(dt.values())
    classes.append(cls)

In [ ]:
np.unique(classes)

# Model

In [ ]:
with tf.device('/cpu:0'):
    img_input = Input(shape=(None, None, 3), name='img_input')
    roi_input = Input(shape=(None, 4), name='roi_input') # 4 == 점의 개수 * 2

    # define the base network (resnet here, can be VGG, Inception, etc)
    shared_layers = nn.nn_base(img_input, trainable=True)

    # define the RPN, built on the base layers
    num_anchors = len(cfg.anchor_box_scales) * len(cfg.anchor_box_ratios) # 3 * 3 == 9
    rpn = nn.rpn(shared_layers, num_anchors) # rpn_out_class 2k이어야 하는데 k 반환
    
    # cfg.num_rois : 32
    classifier = nn.classifier(shared_layers, roi_input, cfg.num_rois, nb_classes=cfg.len_class, trainable=True)

    model_rpn = Model(img_input, rpn[:2], name='model_rpn')
    model_classifier = Model([img_input, roi_input], classifier, name='model_classifier')

    # this is a model that holds both the RPN and the classifier, used to load/save weights for the models
    model_all = Model([img_input, roi_input], rpn[:2] + classifier, name='model_all')

    model_rpn.compile(optimizer=Adam(lr=1e-5), 
                      loss=[losses_fn.rpn_loss_cls(num_anchors), losses_fn.rpn_loss_regr(num_anchors)])
    
    model_classifier.compile(optimizer=Adam(lr=1e-5),
                             loss=[losses_fn.class_loss_cls, losses_fn.class_loss_regr(cfg.len_class - 1)],
                             metrics={'dense_class_{}'.format(cfg.len_class): 'accuracy'})
    
    model_all.compile(optimizer='sgd', loss='mae')

In [ ]:
# plot_model(model_all, to_file=cfg.visual_model_path, show_shapes=True, \
#            show_layer_names=True, expand_nested=False, dpi=200)

# display(Image.open(cfg.visual_model_path))
# model_all.summary(line_length=120)

# Train

In [ ]:
train_length = len(train_images)
val_length = len(validation_images)
num_epochs = int(cfg.num_epochs)
print(num_epochs)

losses = np.zeros((train_length, 5))
val_losses = np.zeros((val_length, 5))
best_loss = np.Inf

rpn_accuracy_for_epoch = []

class_mapping_inv = {v: k for k, v in cfg.class_mapping.items()}

In [ ]:
with tf.device('/cpu:0'):
    for epoch_num in range(num_epochs):
        print('Epoch {}/{}'.format(epoch_num + 1, num_epochs))

        start_time = time.time()
        rpn_accuracy_for_epoch = []
        progbar = Progbar(train_length)
        
        for iter_num in range(train_length):
            X, Y, img_data, cache = next(data_gen_train)
            
            loss_rpn = model_rpn.train_on_batch(X, Y)
            print(f"\t rpn loss -> cls: {loss_rpn[1]}, regr: {loss_rpn[2]}")
            P_rpn = model_rpn.predict_on_batch(X) # (rpn cls, rpn regr)
            result = roi_helpers.rpn_to_roi(P_rpn[0], P_rpn[1], cfg, 'tf', use_regr=True,
                                            overlap_thresh=0.7,
                                            max_boxes=256) # origin code is 300 but paper is set by 256 (128 - positive, 128 - negative)

            # note: calc_iou converts from (x1,y1,x2,y2) to (x,y,w,h) format
            X2, Y1, Y2, IouS = roi_helpers.calc_iou(result, img_data, cfg, cfg.class_mapping)
            
            if X2 is None:
                rpn_accuracy_for_epoch.append(0)
                continue

            pos_samples = np.where(Y1[0, :, -1] == 0)
            neg_samples = np.where(Y1[0, :, -1] == 1)
            rpn_accuracy_for_epoch.append((len(pos_samples)))
            
            if len(neg_samples) > 0:
                neg_samples = neg_samples[0]
            else:
                neg_samples = []

            if len(pos_samples) > 0:
                pos_samples = pos_samples[0]
            else:
                pos_samples = []
            
            if len(pos_samples) < cfg.num_rois // 2:
                selected_pos_samples = pos_samples.tolist()
            else:
                selected_pos_samples = np.random.choice(pos_samples, cfg.num_rois // 2, replace=False).tolist()

            if len(neg_samples) + len(selected_pos_samples) > cfg.num_rois:
                selected_neg_samples = np.random.choice(neg_samples, cfg.num_rois - len(selected_pos_samples), replace=False).tolist()
            else:
                selected_neg_samples = np.random.choice(neg_samples, cfg.num_rois - len(selected_pos_samples), replace=True).tolist()
            selected_samples = selected_pos_samples + selected_neg_samples
            
            loss_class = model_classifier.train_on_batch([X, X2[:, selected_samples, :]], [Y1[:, selected_samples, :], Y2[:, selected_samples, :]])
            print(f"\t cls loss -> cls: {loss_class[1]}, regr: {loss_class[2]}, cls acc: {loss_class[3]}")
            
            #### loss 계산 갱신
            
            losses[iter_num, 0] = loss_rpn[1]
            losses[iter_num, 1] = loss_rpn[2]

            losses[iter_num, 2] = loss_class[1]
            losses[iter_num, 3] = loss_class[2]
            losses[iter_num, 4] = loss_class[3]
            
            rpn_cls = np.nanmean(losses[:iter_num+1, 0])
            rpn_regr = np.nanmean(losses[:iter_num+1, 1])
            detector_cls = np.nanmean(losses[:iter_num+1, 2])
            detector_regr = np.nanmean(losses[:iter_num+1, 3])

            progbar.update(iter_num,
                           [('rpn_cls', rpn_cls), \
                            ('rpn_regr', rpn_regr), \
                            ('detector_cls', detector_cls), \
                            ('detector_regr', detector_regr)])
            model_rpn.save_weights(os.path.join('models', f'Rpn_Epoch_{epoch_num}_Iter_{iter_num:04}_Cls_{rpn_cls:.4f}_Regr_{rpn_regr:.4f}.hdf5'))
            model_classifier.save_weights(os.path.join('models', f'Cls_Epoch_{epoch_num}_Iter_{iter_num:04}_Cls_{detector_cls:.4f}_Regr_{detector_regr:.4f}.hdf5'))
            model_all.save_weights(os.path.join('models', f'Epoch_{epoch_num}_Iter_{iter_num:04}_rpnCls_{rpn_cls:.4f}_rpnRegr_{rpn_regr:.4f}_clsCls_{detector_cls:.4f}_clsRegr_{detector_regr:.4f}.hdf5'))

        loss_rpn_cls = np.nanmean(losses[:, 0])
        loss_rpn_regr = np.nanmean(losses[:, 1])
        loss_class_cls = np.nanmean(losses[:, 2])
        loss_class_regr = np.nanmean(losses[:, 3])
        class_acc = np.nanmean(losses[:, 4])

        mean_overlapping_bboxes = float(sum(rpn_accuracy_for_epoch)) / len(rpn_accuracy_for_epoch)

        if cfg.verbose:
            print('Mean number of bounding boxes from RPN overlapping ground truth boxes: {}'.format(mean_overlapping_bboxes))
            if mean_overlapping_bboxes == 0:
                print('RPN is not producing bounding boxes that overlap the ground truth boxes.')
                print('Check RPN settings or keep training.')
                
            print('Classifier accuracy for bounding boxes from RPN: {}'.format(class_acc))
            print('Loss RPN classifier: {}'.format(loss_rpn_cls))
            print('Loss RPN regression: {}'.format(loss_rpn_regr))
            print('Loss Detector classifier: {}'.format(loss_class_cls))
            print('Loss Detector regression: {}'.format(loss_class_regr))
            print('Elapsed time: {}'.format(time.time() - start_time))

        curr_loss = loss_rpn_cls + loss_rpn_regr + loss_class_cls + loss_class_regr

        if curr_loss < best_loss:
            print(f'\t Total loss decreased from {best_loss:.5f} to {curr_loss:.5f}')
            best_loss = curr_loss
        
        model_rpn.save_weights(os.path.join('models', f'Rpn_Epoch_{epoch_num}_Cls_{loss_rpn_cls:.4f}_Regr_{loss_rpn_regr:.4f}.hdf5'))
        model_classifier.save_weights(os.path.join('models', f'Cls_Epoch_{epoch_num}_Cls_{loss_class_cls:.4f}_Regr_{loss_class_regr:.4f}.hdf5'))
        model_all.save_weights(os.path.join('models', f'Epoch_{epoch_num}_rpnCls_{loss_rpn_cls:.4f}_rpnRegr_{loss_rpn_regr:.4f}_clsCls_{loss_class_cls:.4f}_clsRegr_{loss_class_regr:.4f}_acc{class_acc:.4f}.hdf5'))

---

### Check
- this notebook dosen't use Validation
- if you want to use, loading and evalutating it after learning

### Start
- 2021-03-27 21:39

### Problem
- beginning, nan comes out

 ./save_dota_aug_train/P1848_augment.png (2224, 1584, 3)
#calc_rpn Exception : can't multiply sequence by non-int of type 'float'

 ./save_dota_aug_train/P0282_augment.png (944, 1200, 3)
	 rpn loss -> cls: 7.8367462158203125, regr: 1.8953641653060913
	 cls loss -> cls: 2.9444386959075928, regr: 0.0, cls acc: 0.0
0/5 [..............................] - ETA: 0s - rpn_cls: nan - rpn_regr: nan - detector_cls: nan - detector_regr: nan

---